In [1]:
# -*- coding: utf-8
# The Intercept Brasil (https://theintercept.com/brasil/)
# Reinaldo Chaves (@paidatocandeira)
# A partir dos endereços de empresas gera latitude e longitude para armazenar e baixa imagens do Google Street View
#

In [1]:
import pandas as pd
import googlemaps
from datetime import datetime
import google_streetview.api
import os

Função para checar tamanho de CNPJ

In [11]:
def compara(linha):
    if len(linha['cnpj']) < 14:
        conta = 14 - (len(linha['cnpj']))
        zero = '0'
        zeros = zero * conta
        alterado = zeros + linha['cnpj']
    else:
        alterado = linha['cnpj']
    return alterado

Locais para chaves do Google:
https://developers.google.com/maps/documentation/javascript/get-api-key 

https://developers.google.com/maps/documentation/streetview/get-api-key

## Testes

In [2]:
gmaps = googlemaps.Client(key='')

Geocodificando um endereço

In [14]:
geocode_result = gmaps.geocode('Rua Afonso Cavalcanti, 455 - Cidade Nova, Rio de Janeiro')
#geocode_result = gmaps.geocode('Viaduto do Chá, 15, São Paulo')

In [15]:
geocode_result

[{'address_components': [{'long_name': '455',
    'short_name': '455',
    'types': ['street_number']},
   {'long_name': 'Rua Afonso Cavalcanti',
    'short_name': 'R. Afonso Cavalcanti',
    'types': ['route']},
   {'long_name': 'Cidade Nova',
    'short_name': 'Cidade Nova',
    'types': ['political', 'sublocality', 'sublocality_level_1']},
   {'long_name': 'Rio de Janeiro',
    'short_name': 'Rio de Janeiro',
    'types': ['administrative_area_level_2', 'political']},
   {'long_name': 'Rio de Janeiro',
    'short_name': 'RJ',
    'types': ['administrative_area_level_1', 'political']},
   {'long_name': 'Brazil',
    'short_name': 'BR',
    'types': ['country', 'political']},
   {'long_name': '20211-110',
    'short_name': '20211-110',
    'types': ['postal_code']}],
  'formatted_address': 'R. Afonso Cavalcanti, 455 - Cidade Nova, Rio de Janeiro - RJ, 20211-110, Brazil',
  'geometry': {'location': {'lat': -22.9112524, 'lng': -43.2056333},
   'location_type': 'ROOFTOP',
   'viewport': 

In [16]:
lat = geocode_result[0]['geometry']['location']['lat']

In [17]:
lat

-22.9112524

In [18]:
lng = geocode_result[0]['geometry']['location']['lng']

In [19]:
lng

-43.2056333

Definir parâmetros para o street view api

In [20]:
params = [{
  'size': '600x300', # max 640x640 pixels
  'location': str(lat)+","+str(lng),
  'heading': '0;90;180;270',
  'pitch': '-0.76',
  'key': ''
}]

Cria um objeto de resultados

In [21]:
results = google_streetview.api.results(params)

Download da imagem para diretório 'downloads'

In [22]:
results.download_links('downloads')

Renomeia o arquivo

In [23]:
novo = "downloads/novo1" + ".jpg"

In [24]:
os.rename("downloads/gsv_0.jpg", novo)

# Procura empresas de segurança para fazer o mesmo processo

In [4]:
def mapas_img(cnpj, address, empresa, completo):
    try:
        geocode_result = gmaps.geocode(address)
    except:
        print("Erro em: ", cnpj)
        return

    try:
        lat = geocode_result[0]['geometry']['location']['lat']
    except:
        print("Erro em: ", cnpj)
        return

    try:
        lng = geocode_result[0]['geometry']['location']['lng']
    except:
        print("Erro em: ", cnpj)
        return

    new_df.append({'cnpj': cnpj, 'nome_empresa': empresa, 'latitude': lat, 'longitude': lng})

    # Define parameters for street view api
    params = [{
      'size': '600x300', # max 640x640 pixels
      'location': str(lat)+","+str(lng),
      'heading': '0;90;180;270',
      'pitch': '-0.76',
      'key': ''
            }]

    # Create a results object
    results = google_streetview.api.results(params)

    # Download images to directory 'downloads'
    results.download_links('downloads' + "/" + cnpj + "_" + empresa)

    # Novo nome
    #novo = "downloads/" + "cnpj_" + cnpj + ".jpg"
    #os.rename("downloads/gsv_0.jpg", novo)

    nome = "endereco.txt"

    with open("downloads/" + "/" + cnpj + "_" + empresa + "/" + nome, "a") as arq:
              arq.write(completo)

    return

In [ ]:
gmaps = googlemaps.Client(key='')

policiais = pd.read_csv("policiais_consolidado.csv",sep=',',encoding = 'utf-8', converters={'cnpj': lambda x: str(x)})
policiais = policiais[(policiais['municipio'] == 'DUQUE DE CAXIAS') | (policiais['municipio'] == 'NITEROI') | (policiais['municipio'] == 'NOVA IGUACU') |  (policiais['municipio'] == 'RIO DE JANEIRO') | (policiais['municipio'] == 'SAO GONCALO') | (policiais['municipio'] == 'SAO JOAO DE MERITI')]

new_df = []
compara =  ''
for num, row in policiais.iterrows():
    cnpj = row['cnpj']
    if cnpj != compara:
        address = row['logradouro'] + ", " + str(row['numero_logradouro']) + ", " + ", " + row['municipio']
        completo = row['logradouro'] + ", " + str(row['numero_logradouro']) + ", " + str(row['complemento']) + ", " + row['bairro'] + ", " + row['municipio']
        print(address)
        empresa = row['nome_empresa'].strip()
        mapas_img(cnpj, address, empresa, completo)
        compara = row['cnpj']

In [6]:
df_localizacao1 = pd.DataFrame(new_df, columns = ['cnpj', 'latitude', 'longitude'])

In [7]:
df_localizacao1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159 entries, 0 to 158
Data columns (total 3 columns):
cnpj         159 non-null object
latitude     159 non-null float64
longitude    159 non-null float64
dtypes: float64(2), object(1)
memory usage: 3.8+ KB


In [8]:
df_localizacao1.to_csv('localizacoes_empresas_policiais.csv', index=False)

In [9]:
loca_empre = pd.read_csv("localizacoes_empresas_policiais.csv",sep=',',encoding = 'utf-8', converters={'cnpj': lambda x: str(x)})

Prepara arquivo para o CARTO

In [10]:
carto = pd.merge(policiais.drop_duplicates('cnpj'), loca_empre, left_on='cnpj', right_on='cnpj')

In [11]:
carto.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 159 entries, 0 to 158
Data columns (total 32 columns):
cnpj                                159 non-null object
CNAE_principal                      149 non-null object
nome_empresa                        159 non-null object
nome_empresa_maiuscula_minuscula    131 non-null object
estado                              131 non-null object
indicador                           150 non-null float64
documento                           150 non-null object
qualificacao                        150 non-null object
socio                               153 non-null object
socio_maiuscula_minuscula           134 non-null object
data_abertura                       159 non-null object
logradouro                          159 non-null object
numero_logradouro                   159 non-null object
complemento                         89 non-null object
cep                                 114 non-null object
bairro                              159 non-null object

In [12]:
carto.to_csv('arquivo_carto.csv', index=False)